In [6]:
import numpy as np
np.random.seed(42)

# Decission Tree Interface

In [11]:
from typing import Optional, Literal


class Node:

    def __init__(
        self,
        feature: int, 
        threshold: np.number,
        preds: Optional[dict] = None,
        left: Optional['Node'] = None,
        right: Optional['Node'] = None,
    ):
        self.feature = feature
        self.threshold = threshold
        self.preds = preds
        self.left = left
        self.right = right


class Tree:
    
    def __init__(
        self,
        max_depth: int = 5, 
        min_samples_split: int = 2, 
        min_samples_leaf: int = 2,
        max_features: Optional[int] = None,
    ):
        self.tree = None
        self.criterion = lambda a, b=None: None
        self.prediction_func = lambda _: None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features


    def information_gain(
        self,
        x: np.array, 
        y: np.array, 
        criterion_idx: int, 
        criterion_threshold: np.number,
        sample_weight: Optional[np.array] = None,
    ):
        parent_crit_val = self.criterion(y, sample_weight)

        left_mask = x[:, criterion_idx] < criterion_threshold
        right_mask = ~left_mask

        left_y = y[left_mask]
        right_y = y[right_mask]

        left_w = sample_weight[left_mask] if sample_weight is not None else None
        right_w = sample_weight[right_mask] if sample_weight is not None else None

        left_crit_val = self.criterion(left_y, left_w)
        right_crit_val = self.criterion(right_y, right_w)

        return len(y) * parent_crit_val - len(left_y) * left_crit_val - len(right_y) * right_crit_val


    def split_node(
            self, 
            x: np.array, 
            y: np.array, 
            depth: int, 
            used: set, 
            sample_weight: Optional[np.array] = None,
        ):
        n_samples, n_features = x.shape

        preds = self.prediction_func(y)

        if (n_samples < self.min_samples_split) or (depth >= self.max_depth):
            return preds
        
        best_info_gain, best_feature, best_threshold = -np.inf, None, None
        
        features = set(range(n_features)) - used
        if self.max_features is not None:
            features = np.random.choice(list(features), size=self.max_features)
        
        for feature in features:
            col_vals = np.unique(x[:, feature])
            thresholds = (col_vals[:-1] + col_vals[1:]) / 2
            for threshold in thresholds:
                tmp_x = x[x[:, feature] < threshold]
                if (len(tmp_x) < self.min_samples_leaf) or (len(x) - len(tmp_x) < self.min_samples_leaf):
                    continue

                gain = self.information_gain(
                    x, y, criterion_idx=feature, criterion_threshold=threshold, sample_weight=sample_weight,
                )
                
                if gain > best_info_gain:
                    best_info_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        
        if best_info_gain in [0, -np.inf]:
            return preds
        
        mask = x[:, best_feature] < best_threshold
        sw_left = sample_weight[mask] if sample_weight is not None else None
        sw_right = sample_weight[~mask] if sample_weight is not None else None

        left_node = self.split_node(x[mask], y[mask], depth + 1, used | {best_feature}, sw_left)
        right_node = self.split_node(x[~mask], y[~mask], depth + 1, used | {best_feature}, sw_right)
        
        return Node(
            feature=best_feature, 
            threshold=best_threshold, 
            preds=preds,
            left=left_node, 
            right=right_node, 
        )


    def fit(self, x: np.array, y: np.array, sample_weight: Optional[np.array] = None):
        self.tree = self.split_node(x, y, depth=0, used=set(), sample_weight=sample_weight)

# Decission Tree for classification

In [8]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report


ds = load_breast_cancer()

clf_x_tr, clf_x_test, clf_y_tr, clf_y_test = train_test_split(ds["data"], ds["target"], test_size=0.2, random_state=42, shuffle=True)

### Ручная реализация на np

In [12]:
class ClfDecissionTree(Tree):

    def __init__(
        self, 
        criterion: Literal["gini", "entropy"],
        max_depth: int = 5, 
        min_samples_split: int = 2, 
        min_samples_leaf: int = 2,
        max_features: Optional[int] = None,
    ):
        super().__init__(
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            min_samples_split=min_samples_split,
            max_features=max_features,
        )

        if criterion == "gini":
            self.criterion = ClfDecissionTree.gini
        elif criterion == "entropy":
            self.criterion = ClfDecissionTree.entropy
        else:
            raise ValueError(criterion)
        
        self.prediction_func = self._calc_probs


    @staticmethod
    def _calc_weighted_probs(y: np.array, sample_weight: np.array):
        if (sample_weight is None) or (np.sum(sample_weight) == 0):
            _, counts = np.unique(y, return_counts=True)
            return counts / np.sum(counts)
        else:
            classes = np.unique(y)
            sum_weights = np.sum(sample_weight)
            return np.array([np.sum(sample_weight[y == cl]) / sum_weights for cl in classes])
        

    @staticmethod
    def entropy(y: np.array, sample_weight: Optional[np.array] = None):
        p = ClfDecissionTree._calc_weighted_probs(y, sample_weight)
        
        return -np.sum(p * np.log2(p))
    

    @staticmethod
    def gini(y: np.array, sample_weight: Optional[np.array] = None):
        p = ClfDecissionTree._calc_weighted_probs(y, sample_weight)
            
        return 1 - np.sum(p ** 2)
    

    def _calc_probs(self, y: np.array):
        classes, counts = np.unique(y, return_counts=True)
        probs_dict = dict(zip(classes, counts / len(y)))

        return [probs_dict.get(c, np.float64(0.)) for c in self.classes]


    def fit(self, x: np.array, y: np.array, sample_weight: Optional[np.array] = None):
        self.classes = sorted(np.unique(y))
        self.tree = self.split_node(x, y, depth=0, used=set(), sample_weight=sample_weight)
    
    
    def predict_proba(self, x: np.array):
        preds = []
        for x_obj in x:
            node = self.tree 
            while isinstance(node, Node):
                node = node.left if x_obj[node.feature] < node.threshold else node.right
            
            preds.append(node)

        return np.array(preds)
    
    
    def predict(self, x: np.array):
        probs = self.predict_proba(x)
        inds = np.argmax(probs, axis=-1)

        return np.array(self.classes)[inds]

In [13]:
tree = ClfDecissionTree(
    criterion="gini",
    max_depth=3, 
    min_samples_split=10, 
    min_samples_leaf=10,
    max_features=10,
)
tree.fit(clf_x_tr, clf_y_tr)

print("Результаты на test:\n\n", classification_report(clf_y_test, tree.predict(clf_x_test)))
print("ROC-AUC на test:", roc_auc_score(clf_y_test, tree.predict_proba(clf_x_test)[:, 1]).round(3))

Результаты на test:

               precision    recall  f1-score   support

           0       1.00      0.93      0.96        43
           1       0.96      1.00      0.98        71

    accuracy                           0.97       114
   macro avg       0.98      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

ROC-AUC на test: 0.97


In [14]:
weights = np.ones_like(clf_y_tr)
weights[clf_y_tr == 0] = 4

tree = ClfDecissionTree(
    criterion="gini",
    max_depth=3, 
    min_samples_split=10, 
    min_samples_leaf=10,
    max_features=10,
)
tree.fit(clf_x_tr, clf_y_tr, sample_weight=weights)

print("Результаты на test:\n\n", classification_report(clf_y_test, tree.predict(clf_x_test)))
print("ROC-AUC на test:", roc_auc_score(clf_y_test, tree.predict_proba(clf_x_test)[:, 1]).round(3))

Результаты на test:

               precision    recall  f1-score   support

           0       0.86      1.00      0.92        43
           1       1.00      0.90      0.95        71

    accuracy                           0.94       114
   macro avg       0.93      0.95      0.94       114
weighted avg       0.95      0.94      0.94       114

ROC-AUC на test: 0.948


### Сравнение с sklearn

In [45]:
from sklearn.tree import DecisionTreeClassifier


sklearn_tree = DecisionTreeClassifier(
    criterion="gini", 
    max_depth=3, 
    min_samples_split=10, 
    min_samples_leaf=10,
    max_features=10,
)
sklearn_tree.fit(clf_x_tr, clf_y_tr)

DecisionTreeClassifier(max_depth=3, max_features=10, min_samples_leaf=10,
                       min_samples_split=10)

In [46]:
print("Результаты на test:\n\n", classification_report(clf_y_test, sklearn_tree.predict(clf_x_test)))
print("ROC-AUC на test:", roc_auc_score(clf_y_test, sklearn_tree.predict_proba(clf_x_test)[:, 1]).round(3))

Результаты на test:

               precision    recall  f1-score   support

           0       0.95      0.95      0.95        43
           1       0.97      0.97      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114

ROC-AUC на test: 0.982


# Ensembling 

! Aнсамблирование — это общий термин, обозначающий любую технику, которая создаёт прогнозы путём объединения прогнозов нескольких моделей.

Ансамблевые подходы часто могут повысить производительность одной модели. Ансамблевый подход может помочь снизить:

- Дисперсия путем усреднения нескольких моделей

- Предвзятость, возникающая при итеративном исправлении ошибок

- Переобучение, поскольку использование нескольких моделей может повысить устойчивость к ложным связям

! Почему ансамбль не переобучается, как дерево?

Одно дерево — склонно к переобучению, если дать ему много свободы. В ансамбле каждое дерево переобучается по-своему, но их ошибки не совпадают. При усреднении они взаимно гасятся, а верные сигналы остаются. Статистически в среднем, если у каждого дерева вероятность угадать класс > 0.5, то вместе в среднем чем больше деревьев, тем выше вероятность правильного предсказания. Даже если отдельные деревья переобучены — их ошибки не коррелируют, и итоговое предсказание сглаживает шум.

### Bagging и Random Forest

Бэггинг — это особый метод ансамблевого обучения, который используется для уменьшения дисперсии прогностической модели. Дисперсия в смысле машинного обучения - насколько модель меняется при изменении обучающего набора данных. Поскольку бэггинг помогает уменьшить дисперсию модели машинного обучения, он часто улучшает модели с высокой дисперсией (например, деревья решений и KNN), но малоэффективен для моделей с низкой дисперсией (например, линейная регрессия).


Случайный лес - на каждом разбиении дерева выбирается случайное подмножество признаков для лучшего разделения. Дополнительная случайность (выбор признаков) снижает корреляцию между деревьями. Фичи случайны — деревья не просто по-разному обучаются, но и по-разному "смотрят" на данные. Это усиливает разнообразие.

In [48]:
class ClfRandomForest:

    def __init__(self, n_trees: int, **tree_kwargs):
        self.n_trees = n_trees
        self.trees = []
        self.tree_params = tree_kwargs

    
    def fit(self, x: np.array, y: np.array, bootstrap_part: float = 0.5):
        self.classes = sorted(np.unique(y))

        bootstrap_size = int(len(y) * bootstrap_part)

        for _ in range(self.n_trees):
            idx = np.random.randint(0, len(y), size=(bootstrap_size, ))
            tree = ClfDecissionTree(**self.tree_params)
            tree.fit(x[idx], y[idx])
            tree.classes = self.classes
            self.trees.append(tree)
    

    def predict(self, x: np.array):
        preds = []
        for tree in self.trees:
            preds.append(tree.predict(x))
        
        probs = []
        for results in np.c_[*preds]:
            classes, counts = np.unique(results, return_counts=True)
            probs_dict = dict(zip(classes, counts / self.n_trees))

            probs.append([probs_dict.get(c, np.float64(0.)) for c in self.classes])
        
        return np.array(probs)

In [49]:
bag = ClfRandomForest(
    n_trees=10, 
    **dict(
        criterion="gini",
        max_depth=3, 
        min_samples_split=10, 
        min_samples_leaf=10,
        max_features=10,  # Если None - Bagging, иначе - Random Forest.
    )
)
bag.fit(clf_x_tr, clf_y_tr)

In [50]:
roc_auc_score(clf_y_test, bag.predict(clf_x_test)[:, 1])

np.float64(0.9931215198165739)

### AdaBoost на np

Бустинг - метод ансамблевого обучения. В отличие от бэггинга модели зависят друг от друга. Несколько слабых классификаторов обучаются друг за другом, каждый сосредотачивается на ошибках предыдущих.

In [15]:
class AdaBoost:

    def __init__(self, **tree_kwargs):
        self.clfs = []
        self.alphas = []
        self.tree_params = tree_kwargs
    

    def fit(self, x: np.array, y: np.array, n_clfs: int):
        assert set(np.unique(y)).issubset({-1, 1}), "classes must be only {-1; 1}"
        
        n_samples = len(x)
        w = np.ones(n_samples, dtype=float) / n_samples

        for _ in range(n_clfs):
            # Обучение пня
            tree = ClfDecissionTree(**self.tree_params)
            tree.fit(x, y, sample_weight=w)
            
            # Расчет взвешенной ошибки
            preds = tree.predict(x)
            misclassified = (preds != y)
            err = np.sum(w * misclassified)

            # Если дерево работает хуже случайного предсказателя
            if err >= 0.5:
                break
            
            # Вес модели
            alpha = 0.5 * np.log((1 - err) / err)

            # Обновление весов объектов
            w *= np.exp(-alpha * y * preds)
            w /= np.sum(w)

            # Сохранение пня
            self.alphas.append(alpha)
            self.clfs.append(tree)

    
    def _tmp_predict(self, x: np.array):
        preds = np.zeros(shape=(x.shape[0], ))
        for alpha, model in zip(self.alphas, self.clfs):
            preds += alpha * model.predict(x)

        return preds
    

    def predict(self, x: np.array):
        return np.sign(self._tmp_predict(x))
    
    
    def predict_proba(self, x: np.array):
        logits = self._tmp_predict(x)

        probs_pos = 1 / (1 + np.exp(-2 * logits))
        probs_neg = 1 - probs_pos

        return np.c_[probs_neg, probs_pos]

In [16]:
ab = AdaBoost(**dict(criterion="gini", max_depth=4, min_samples_leaf=10, min_samples_split=10, max_features=10))

ab_y_tr = np.copy(clf_y_tr)
ab_y_tr[ab_y_tr == 0] = -1
ab.fit(clf_x_tr, ab_y_tr, n_clfs=50)

ab_y_test = np.copy(clf_y_test)
ab_y_test[ab_y_test == 0] = -1
print("Результаты на test:\n\n", classification_report(ab_y_test, ab.predict(clf_x_test)))
print("ROC-AUC на test:", roc_auc_score(ab_y_test, ab.predict_proba(clf_x_test)[:, 1]).round(3))

Результаты на test:

               precision    recall  f1-score   support

          -1       0.95      0.93      0.94        43
           1       0.96      0.97      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114

ROC-AUC на test: 0.957


# Decission Tree for regression

In [31]:
from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_squared_error


ds = load_diabetes(scaled=False)

reg_x_tr, reg_x_test, reg_y_tr, reg_y_test = train_test_split(ds["data"], ds["target"], test_size=0.2, random_state=42, shuffle=True)

### Ручная реализация на np

In [32]:
class RegDecissionTree(Tree):

    def __init__(
        self, 
        criterion: Literal["mae", "mse"],
        max_depth: int = 5, 
        min_samples_split: int = 2, 
        min_samples_leaf: int = 2,
    ):
        super().__init__(
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            min_samples_split=min_samples_split,
        )

        if criterion == "mse":
            self.criterion = RegDecissionTree.mse
            self.prediction_func = RegDecissionTree._predict_mean
        elif criterion == "mae":
            self.criterion = RegDecissionTree.mae
            self.prediction_func = RegDecissionTree._predict_median
        else:
            raise ValueError(criterion)


    @staticmethod
    def mse(y: np.array):
        return np.sum((y - np.mean(y)) ** 2)
    

    @staticmethod
    def mae(y: np.array):
        return np.sum(np.abs(y - np.median(y)))
    
    @staticmethod
    def _predict_mean(y: np.array):
        return np.mean(y)
    

    @staticmethod
    def _predict_median(y: np.array):
        return np.median(y)

    
    def predict(self, x: np.array):
        preds = []
        for x_obj in x:
            node = self.tree 
            while isinstance(node, Node):
                node = node.left if x_obj[node.feature] < node.threshold else node.right
            
            preds.append(node)

        return np.array(preds)

In [33]:
tree = RegDecissionTree(
    criterion="mse",
    max_depth=10, 
    min_samples_split=5, 
    min_samples_leaf=5,
)
tree.fit(reg_x_tr, reg_y_tr)

In [34]:
mean_squared_error(reg_y_test, tree.predict(reg_x_test))

3796.5563812188416

### Сравнение с sklearn

In [35]:
from sklearn.tree import DecisionTreeRegressor


sklearn_tree = DecisionTreeRegressor(
    criterion="squared_error", 
    max_depth=10, 
    min_samples_split=5, 
    min_samples_leaf=5,
)
sklearn_tree.fit(reg_x_tr, reg_y_tr)

DecisionTreeRegressor(max_depth=10, min_samples_leaf=5, min_samples_split=5)

In [36]:
mean_squared_error(reg_y_test, sklearn_tree.predict(reg_x_test))

3400.8392631942497